<a href="https://colab.research.google.com/github/muhammedshelleh/health-data-cleaning/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# svm.py
import numpy as np  # for handling multi-dimensional array operation
import pandas as pd  # for reading data from csv 
import statsmodels.api as sm  # for finding the p-value
from sklearn.preprocessing import MinMaxScaler  # for normalization
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score 
from sklearn.utils import shuffle

In [47]:
def CompletenessRatio(data):
    completeness_ratio = []
    for column in data.columns:
        ratio =  data[column].describe()[0] / data[column].size
        completeness_ratio.append(ratio)
    return completeness_ratio

In [48]:
def InCompletenessRatio(data):
    completeness_ratio = []
    for column in data.columns:
        ratio = 1-  data[column].describe()[0] / data[column].size 
        completeness_ratio.append(ratio)
    return completeness_ratio

In [49]:
def get_best_dist(completeness_column_wise_data):
    # # Distribution Fitting using SciPy
    # ##  Above cell show that the best distribution that the data could be fit to is alpha
    dist_results = []
    for i in dists:
        dist = getattr(stats, i)
        param = dist.fit(completeness_column_wise_data)
        a = stats.kstest(completeness_column_wise_data, i, args=param)
        dist_results.append((i, a[0], a[1]))
    dist_results.sort(key=lambda x: float(x[2]), reverse=True)
    for j in dist_results:
        print("{}: statistic={}, pvalue={}".format(j[0], j[1], j[2]))
    return dist_results[0][0]

In [50]:
def distribution_fitting(sample):
    results_1 = []
    for i in dists:
        dist = getattr(stats, i)
        param = dist.fit(sample)
        a = stats.kstest(sample, i, args=param)
        results_1.append((i,a[0],a[1]))


    results_1.sort(key=lambda x:float(x[2]), reverse=True)
    for j in results_1:
        print("{}: statistic={}, pvalue={}".format(j[0], j[1], j[2]))

In [51]:
def get_fitted_points(input_data):
    best_dist = 'foldcauchy'
    print("Best dist: ", best_dist)
    dist = getattr(stats, best_dist)
    param = dist.fit(input_data)

    x = input_data
    y = stats.foldcauchy.pdf(x, *param)
    return x, y

In [52]:
# >> FEATURE SELECTION << #
def remove_correlated_features(X):
  def remove_less_significant_features(X, Y):
# >> MODEL TRAINING << #
    def compute_cost(W, X, Y):
      def calculate_cost_gradient(W, X_batch, Y_batch):
        def sgd(features, outputs):
          def init():
              n_bins = 10
              data = pd.read_csv('NNDSS_-_TABLE_1FF._Severe_acute_respiratory_syndrome-associated_coronavirus_disease_to_Shigellosis.csv')
              completeness_column_wise = CompletenessRatio(data)
              # SVM only accepts numerical values. 
              # Therefore, we will transform the categories M and B into
              # values 1 and -1 (or -1 and 1), respectively.
              diagnosis_map = {'NaN':0, '1':1}
              data['Shigellosis, Current week'] = data['Shigellosis, Current week'].map(diagnosis_map)
              # drop last column (extra column added by pd)
              # and unnecessary first column (id)
              data.drop(data.columns[[-1, 0]], axis=1, inplace=True)
              Y = data.loc[:, 'Shigellosis, Current week']  # all rows of 'diagnosis' 
              X = data.iloc[:, 1:]  # all rows of column 1 and ahead (features)
              # normalize the features using MinMaxScalar from
              # sklearn.preprocessing
              X_normalized = MinMaxScaler().fit_transform(X.values)
              X = pd.DataFrame(X_normalized)
              # first insert 1 in every row for intercept b
              X.insert(loc=len(X.columns), column='intercept', value=1)
              # test_size is the portion of data that will go into test set
              # random_state is the seed used by the random number generator
              print("splitting dataset into train and test sets...")
              X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)
              print("training started...")
              W = sgd(X_train.to_numpy(), y_train.to_numpy())
              print("training finished.")
              print("weights are: {}".format(W))
              # testing the model on test set
              y_test_predicted = np.array([])
              for i in range(X_test.shape[0]):
                  yp = np.sign(np.dot(W, X_test.to_numpy()[i])) #model
                  y_test_predicted = np.append(y_test_predicted, yp)
              print("accuracy on test dataset: {}".format(accuracy_score(y_test.to_numpy(), y_test_predicted)))
              print("recall on test dataset: {}".format(recall_score(y_test.to_numpy(), y_test_predicted)))
              print("precision on test dataset: {}".format(recall_score(y_test.to_numpy(), y_test_predicted)))

In [53]:
df = pd.read_csv('NNDSS_-_TABLE_1FF._Severe_acute_respiratory_syndrome-associated_coronavirus_disease_to_Shigellosis.csv')
df.head()

,Reporting Area,MMWR Year,MMWR Week,"Severe acute respiratory syndrome-associated coronavirus desease, Current week","Severe acute respiratory syndrome-associated coronavirus desease, Current week, flag","Severe acute respiratory syndrome-associated coronavirus desease, Previous 52 weeks Max†","Severe acute respiratory syndrome-associated coronavirus desease, Previous 52 weeks Max†, flag","Severe acute respiratory syndrome-associated coronavirus desease, Cum 2021†","Severe acute respiratory syndrome-associated coronavirus desease, Cum 2021†, flag","Severe acute respiratory syndrome-associated coronavirus desease, Cum 2020†","Severe acute respiratory syndrome-associated coronavirus desease, Cum 2020†, flag","Shiga toxin-producing Escherichia coli(STEC), Current week","Shiga toxin-producing Escherichia coli(STEC), Current week, flag","Shiga toxin-producing Escherichia coli(STEC), Previous 52 weeks Max†","Shiga toxin-producing Escherichia coli(STEC), Previous 52 weeks Max†, flag","Shiga toxin-producing Escherichia coli(STEC), Cum 2021†","Shiga toxin-producing Escherichia coli(STEC), Cum 2021†, flag","Shiga toxin-producing Escherichia coli(STEC), Cum 2020†","Shiga toxin-producing Escherichia coli(STEC), Cum 2020†, flag","Shigellosis, Current week","Shigellosis, Current week, flag","Shigellosis, Previous 52 weeks Max†","Shigellosis, Previous 52 weeks Max†, flag","Shigellosis, Cum 2021†","Shigellosis, Cum 2021†, flag","Shigellosis, Cum 2020†","Shigellosis, Cum 2020†, flag",Location 1,Location 2,Reporting Area Sort,geocode
0,TOTAL,2021,1,NaN,-,4,NaN,NaN,-,NaN,-,11.0,NaN,221,NaN,11.0,NaN,161.0,NaN,21.0,NaN,336,NaN,21.0,NaN,231.0,NaN,NaN,TOTAL,20210170,NaN
1,US RESIDENTS,2021,1,NaN,-,4,NaN,NaN,-,NaN,-,11.0,NaN,221,NaN,11.0,NaN,161.0,NaN,21.0,NaN,336,NaN,21.0,NaN,231.0,NaN,NaN,US RESIDENTS,20210101,NaN
2,NEW HAMPSHIRE,2021,1,NaN,-,0,NaN,NaN,-,NaN,-,NaN,-,2,NaN,NaN,-,NaN,-,NaN,-,2,NaN,NaN,-,NaN,-,NEW HAMPSHIRE,NaN,20210106,POINT (-71.57139 43.680429)
3,NEW YORK,2021,1,NaN,-,0,NaN,NaN,-,NaN,-,2.0,NaN,21,NaN,2.0,NaN,NaN,-,NaN,-,9,NaN,NaN,-,NaN,-,NEW YORK,NaN,20210111,POINT (-75.59655 42.921241)
4,PACIFIC,2021,1,NaN,-,1,NaN,NaN,-,NaN,-,NaN,-,47,NaN,NaN,-,56.0,NaN,4.0,NaN,84,NaN,4.0,NaN,47.0,NaN,NaN,PACIFIC,20210157,NaN


In [54]:
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = reg_strength * (np.sum(distances) / N)
    
    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

In [55]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    # if only one example is passed (eg. in case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])
    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))
    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (reg_strength * Y_batch[ind] * X_batch[ind])
        dw += di
    dw = dw/len(Y_batch)  # average
    return dw

In [56]:
def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    # stochastic gradient descent
    for epoch in range(1, max_epochs): 
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)
            
    return weights

In [57]:
def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)
        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is:{} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights

In [58]:
# set hyper-parameters and call init
# hyper-parameters are normally tuned using cross-validation
# but following work good enough
reg_strength = 10000 # regularization strength
learning_rate = 0.000001
init()

KeyError: ignored